In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb
import scipy.sparse as sps
import numpy as np
import torch
torch.manual_seed(2020)
from torch import nn
import torch.nn.functional as F
from math import sqrt

def sigmoid(x):
    return 1./(1. + np.exp(-x))

from dataset import load_data, load_features
from matrix_factorization import MF, MF_N_IPS, MF_N_DR_JL, MF_N_MRDR_JL
from baselines import MF, MF_IPS, MF_ASIPS, MF_SNIPS, MF_DR, MF_DR_JL, MF_MRDR_JL, MF_BaseModel
from models import MLP, MLP_exp, MLP_weibull, MLP_lognormal, MF_IPS_DF, MF_DR_JL_DF

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
from utils import ndcg_func_both, ndcg_func_feature, recall_func_both, recall_func_feature, generate_total_sample
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)


dataset_name = "coat"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)
n_train = x_train.shape[0]

train_user_ind = x_train[:, 0].astype('int')
train_item_ind = x_train[:, 1].astype('int')
test_user_ind = x_test[:, 0].astype('int')
test_item_ind = x_test[:, 1].astype('int')

# recover the complete matrix and retrieve the features

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train, 
    lr=0.05,
    lamb=1e-3,
    tol=1e-4)

x_train_x, u_emb_train, v_emb_train = mf.forward(x_train, True)
x_test_x, u_emb_test, v_emb_test = mf.forward(x_test, True)

# for the feature set of training dataset and test dataset
feature_train = torch.cat([u_emb_train, v_emb_train], axis = 1).detach().cpu()
feature_test = torch.cat([u_emb_test, v_emb_test], axis = 1).detach().cpu()

# the feature of user and item
user_W = mf.W(torch.LongTensor(np.arange(num_user) ).to(device) ).detach().cpu()
item_H = mf.H(torch.LongTensor(np.arange(num_item) ).to(device) ).detach().cpu()

# L = 5, 20%, 19.66%
# L = 6.7, 15%, 15.05%
# L = 3.2, 30%, 30.12%
# L = 9.7, 10%, 10.02%
# L = 3.92, 25%, 24.99%

L = 5
sigmaH = 0.1

num_feature = feature_train.shape[1]
identity_p = np.diag(np.ones(num_feature))
mean_p = np.zeros(num_feature)

W_d = np.random.multivariate_normal(mean_p, sigmaH**2*identity_p)
print(W_d)

y_train_mask = np.zeros_like(y_train)
e_train = np.zeros_like(y_train, dtype='float')
d_train = np.zeros_like(y_train, dtype='float') + 1e5

prod = 0
for i in range(n_train):

    ts_i = np.random.uniform(0, L)
    lambda_i = np.exp( np.dot(W_d, feature_train[i, :]) )
    d_i = np.random.exponential(lambda_i)
    e_i = L - ts_i
    if d_i <= e_i:
        y_train_mask[i] = y_train[i]
        d_train[i] = d_i
    else:
        if y_train[i] == 1:
            prod += 1
        y_train_mask[i] = 0
    
    e_train[i] = e_i

print('mask proportion (within the original y_train = 1 )')
print( prod/(sum(y_train) )*100 )

test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print('prediction for base model')
print(auc_mf)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

feature_test = feature_test.to(device)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300
[MF] epoch:8, xent:37.39917492866516
[-0.04494046 -0.09064704  0.11771674 -0.20835941  0.08493842 -0.07913483
 -0.04008389  0.0490871 ]
mask proportion (within the original y_train = 1 )
19.65764770844837
prediction for base model
0.6076920935476612


In [2]:
# for our model
# for IPS
mf_ips_df_acc = []
for repeat in np.arange(10):
    mf_ips_df = MF_IPS_DF(num_user, num_item, num_feature, batch_size=128)
    mf_ips_df.to(device)
    mf_ips_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, y_ips,lr=0.025, lamb=6e-3, lamb1=4e-2, tol=1e-5)
    test_pred = mf_ips_df.predict(x_test, feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_both(mf_ips_df, x_test, y_test, feature_test)
    recall_res = recall_func_both(mf_ips_df, x_test, y_test, feature_test)

    print(auc)

    mf_ips_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_ips_df_acc = np.array(mf_ips_df_acc)
mf_ips_df_mean = mf_ips_df_acc.mean(0)
mf_ips_df_sd = mf_ips_df_acc.std(0)

print("[MF_IPS_DF] test auc:", mf_ips_df_mean[1], ' sd: ', mf_ips_df_sd[1])
print(mf_ips_df_mean)
print(mf_ips_df_sd)

[MF-IPS-DF] epoch:26, xent:546.5608406066895
0.6943252917854649
[MF-IPS-DF] epoch:38, xent:547.4289293289185
0.6955967904952136
[MF-IPS-DF] epoch:31, xent:545.5821886062622
0.6984420515961302
[MF-IPS-DF] epoch:32, xent:546.1637029647827
0.7022001934796881
[MF-IPS-DF] epoch:27, xent:545.771372795105
0.6887865297306828
[MF-IPS-DF] epoch:30, xent:546.2071666717529
0.6980567548151466
[MF-IPS-DF] epoch:28, xent:547.3947734832764
0.6981815461207788
[MF-IPS-DF] epoch:30, xent:547.3904399871826
0.6984907695032089
[MF-IPS-DF] epoch:31, xent:545.4105291366577
0.6974885725575896
[MF-IPS-DF] epoch:32, xent:545.4218397140503
0.7010777483665969
[MF_IPS_DF] test auc: 0.6972646248450501  sd:  0.003558192340641112
[0.23378554 0.69726462 0.60033571 0.67488807 0.40528531 0.69599892]
[0.000365   0.00355819 0.00766265 0.00415612 0.00881216 0.00376149]


In [3]:
# for DR
mf_dr_df_acc = []
for repeat in np.arange(10):
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H,  y_ips, lr=0.03, lamb=8e-2, lambv=8e-3, tol=1e-5)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

    mf_dr_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_df_acc = np.array(mf_dr_df_acc)
mf_dr_df_mean = mf_dr_df_acc.mean(0)
mf_dr_df_sd = mf_dr_df_acc.std(0)

print("[MF_DR_JL_DF] test auc:", mf_dr_df_mean[1], ' sd: ', mf_dr_df_sd[1])
print(mf_dr_df_mean)
print(mf_dr_df_sd)

[MF-DR-JL-DF] epoch:25, xent:51942431.125
0.7190565893289225
[MF-DR-JL-DF] epoch:19, xent:52013820.9375
0.7098946262601893
[MF-DR-JL-DF] epoch:23, xent:51978533.125
0.7160765806834272
[MF-DR-JL-DF] epoch:22, xent:51997090.125
0.723242752824672
[MF-DR-JL-DF] epoch:20, xent:51871583.5625
0.7235610624834223
[MF-DR-JL-DF] epoch:25, xent:51976428.8125
0.7482972898150962
[MF-DR-JL-DF] epoch:22, xent:52033336.75
0.7149237835409258
[MF-DR-JL-DF] epoch:22, xent:52050953.5
0.7275527410009133
[MF-DR-JL-DF] epoch:21, xent:51995406.3125
0.7249548392734382
[MF-DR-JL-DF] epoch:21, xent:51979089.5
0.7449449371654994
[MF_DR_JL_DF] test auc: 0.7252505202376506  sd:  0.01181637680909129
[0.33722414 0.72525052 0.62493526 0.6943029  0.44025551 0.72079609]
[0.00081414 0.01181638 0.02448759 0.0212155  0.02072552 0.01513417]


In [4]:
# for weibull
mlp_weibull_acc = []
for repeat in np.arange(10):
    mlp_weibull = MLP_weibull(num_feature, batch_size=128)
    mlp_weibull.to(device)
    mlp_weibull.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp_weibull.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_weibull, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_weibull, x_test, y_test, feature_test)

    print(auc)

    mlp_weibull_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_weibull_acc = np.array(mlp_weibull_acc)
mlp_weibull_mean = mlp_weibull_acc.mean(0)
mlp_weibull_sd = mlp_weibull_acc.std(0)

print("[MLP_weibull] test auc:", mlp_weibull_mean[1], ' sd: ', mlp_weibull_sd[1])
print(mlp_weibull_mean)
print(mlp_weibull_sd)

[MLP_weibull] epoch:16, xent:46.410068452358246
0.6326865064543494
[MLP_weibull] epoch:17, xent:46.434557139873505
0.6326055032675797
[MLP_weibull] epoch:16, xent:46.48916149139404
0.6326732636899253
[MLP_weibull] epoch:15, xent:46.471764266490936
0.6326715237646724
[MLP_weibull] epoch:19, xent:46.466258347034454
0.63262773564581
[MLP_weibull] epoch:15, xent:46.404775977134705
0.6326380785348128
[MLP_weibull] epoch:17, xent:46.42523318529129
0.6326758735778044
[MLP_weibull] epoch:19, xent:46.45223939418793
0.6326848631916107
[MLP_weibull] epoch:18, xent:46.38938796520233
0.6326654340262876
[MLP_weibull] epoch:17, xent:46.46602326631546
0.6326168127817229
[MLP_weibull] test auc: 0.6326545594934576  sd:  2.8233968718163342e-05
[0.23576738 0.63265456 0.52546572 0.60533727 0.35953379 0.64539805]
[2.66817799e-03 2.82339687e-05 2.92549837e-04 5.06445465e-05
 4.56308810e-04 1.11022302e-16]


In [7]:
# for DR
mf_dr_acc = []
for repeat in np.arange(10):
    mf_dr = MF_DR(num_user, num_item, batch_size=128)
    mf_dr.to(device)
    mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mf_dr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr, x_test, y_test)
    recall_res = recall_func(mf_dr, x_test, y_test)

    print(auc)

    mf_dr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_acc = np.array(mf_dr_acc)
mf_dr_mean = mf_dr_acc.mean(0)
mf_dr_sd = mf_dr_acc.std(0)

print("[MF_DR] test auc:", mf_dr_mean[1], ' sd: ', mf_dr_sd[1])
print(mf_dr_mean)
print(mf_dr_sd)

[MF-DR] epoch:14, xent:57242.03698730469
0.684786538236984
[MF-DR] epoch:16, xent:57271.133544921875
0.6697801662440581
[MF-DR] epoch:13, xent:57202.19366455078
0.6916481267964728
[MF-DR] epoch:15, xent:57207.057861328125
0.6869813572808913
[MF-DR] epoch:15, xent:57224.549072265625
0.6847825750739082
[MF-DR] epoch:13, xent:57183.53820800781
0.6922814595884961
[MF-DR] epoch:15, xent:57257.57403564453
0.6856412281861705
[MF-DR] epoch:13, xent:57191.561767578125
0.692785361274213
[MF-DR] epoch:21, xent:57219.313537597656
0.6837320468712664
[MF-DR] epoch:14, xent:57250.47131347656
0.6784597833677065
[MF_DR] test auc: 0.6850878642920168  sd:  0.00661972175056715
[0.2359489  0.68508786 0.57347221 0.65197622 0.39148532 0.6891084 ]
[0.00018493 0.00661972 0.0102502  0.00829983 0.00770244 0.00700589]
